"""
Load traffic.csv into "traffic" table in sqlite3 database.

Drop and report invalid rows.
- ip should be valid IP (see ipaddress)
- time must not be in the future
- path can't be empty
- status code must be a valid HTTP status code (see http.HTTPStatus)
- size can't be negative or empty

Report the percentage of bad rows. Fail the ETL if there are more than 5% bad rows
"""

In [39]:
import sqlite3 
import pandas as pd
from ipaddress import ip_address
from datetime import datetime

In [25]:
df=pd.read_csv('traffic.csv',parse_dates=['time'])

In [27]:
df.head()

,ip,time,path,status,size
0,108.66.146.1,2017-06-19T14:03:00,/images,200,1095
1,108.66.146.3,2017-06-19T14:03:21,/posts,200,1572
2,108.66.146.6,2017-06-19T14:03:42,/posts,200,1174
3,108.66.146.1,2017-06-19T14:04:03,/users,200,684
4,108.66.146.1,2017-06-19T14:04:24,/images,400,0


In [57]:
# ✅ Convert 'time' to datetime before applying the function
df['time'] = pd.to_datetime(df['time'], errors='coerce')

In [59]:
df.head()

,ip,time,path,status,size
0,108.66.146.1,2017-06-19 14:03:00,/images,200,1095
1,108.66.146.3,2017-06-19 14:03:21,/posts,200,1572
2,108.66.146.6,2017-06-19 14:03:42,/posts,200,1174
3,108.66.146.1,2017-06-19 14:04:03,/users,200,684
4,108.66.146.1,2017-06-19 14:04:24,/images,400,0


In [29]:
df.info

<bound method DataFrame.info of                ip                 time     path  status  size
0    108.66.146.1  2017-06-19T14:03:00  /images     200  1095
1    108.66.146.3  2017-06-19T14:03:21   /posts     200  1572
2    108.66.146.6  2017-06-19T14:03:42   /posts     200  1174
3    108.66.146.1  2017-06-19T14:04:03   /users     200   684
4    108.66.146.1  2017-06-19T14:04:24  /images     400     0
..            ...                  ...      ...     ...   ...
123  108.66.146.5  2017-06-19T14:46:03   /posts     200  1750
124  108.66.146.3  1017-06-19T14:46:24   /users     200  1079
125  108.66.146.1  2017-06-19T14:46:45  /images     200  1064
126  108.66.146.5  2017-06-19T14:47:06  /health     200  1531
127  108.66.146.4  2017-06-19T14:47:27  /images     200  1029

[128 rows x 5 columns]>

In [ ]:
# def is_valid_ipv4(ip):
#     pattern = r'^(\d{1,3}\.){3}\d{1,3}$'
#     if not re.match(pattern, ip):
#         return False
#     parts = ip.split('.')
#     return all(0 <= int(part) <= 255 for part in parts)

# df['ip_valid'] = df['ip'].apply(is_valid_ipv4)


In [61]:
def is_valid_row(row):
    # 1. Validate IP address
    try:
        ip_address(row['ip'])
    except ValueError:
        return False
     # 2. Time must not be in the future
    # if row['time'] > pd.Timestamp.now():
    # if pd.isna(row['time']) or row['time'] > pd.Timestamp.now():
    now = pd.Timestamp.now()
    if row['time'] > now:

        return False
    # 3. Path must not be empty    
    if pd.isna(row['path']) or str(row['path']).strip() == '':
        return False
    # 4. Status must be a valid HTTP status code
    try:
        if int(row['status']) not in HTTPStatus._value2member_map_:
            return False
    except:
        return False
    # 5. Size must be non-negative and not empty
    if pd.isna(row['size']) or row['size'] < 0:
        return False
    
    return True



In [63]:
# Filter valid and invalid rows
valid_rows = df[df.apply(is_valid_row, axis=1)]
invalid_rows = df[~df.index.isin(valid_rows.index)]

# Report bad rows
total_rows = len(df)
bad_rows_count = len(invalid_rows)
bad_percent = bad_rows_count / total_rows * 100

print(f"{bad_percent:.2f}% bad rows")
if bad_rows_count > 0:
    print("Invalid rows:")
    print(invalid_rows)

# Fail ETL if more than 5% bad rows
if bad_percent > 5:
    raise Exception("ETL failed due to too many bad rows.")

3.91% bad rows
Invalid rows:
              ip                time     path  status  size
10  108.66.446.2 2017-06-19 14:06:30   /users     200   833
17  108.66.146.1 2037-06-19 14:08:57   /users     200  1658
21  108.66.146.2 2017-06-19 14:10:21      NaN     200  1987
26  108.66.146.3 2017-06-19 14:12:06   /posts     200 -1755
59  108.66.146.4 2017-06-19 14:23:39  /images     900  1426


In [65]:
# Save to SQLite
conn = sqlite3.connect("traffic.db")

schema = '''
CREATE TABLE IF NOT EXISTS traffic (
    ip TEXT,
    time TEXT,
    path TEXT,
    status INTEGER,
    size INTEGER
);
'''

conn.executescript(schema)
valid_rows.to_sql("traffic", conn, if_exists="append", index=False)
conn.close()

print("✅ ETL complete: valid rows inserted into SQLite.")

✅ ETL complete: valid rows inserted into SQLite.


In [ ]:
"""
Load traffic.csv into "traffic" table in sqlite3 database.

Drop and report invalid rows.
- ip should be valid IP (see ipaddress)
- time must not be in the future
- path can't be empty
- status code must be a valid HTTP status code (see http.HTTPStatus)
- size can't be negative or empty

Report the percentage of bad rows. Fail the ETL if there are more than 5% bad rows
"""


import sqlite3
from contextlib import closing
from http import HTTPStatus
from ipaddress import ip_address

import pandas as pd

status_codes = set(HTTPStatus)

max_bad_percent = 5


def is_valid_row(row):
    # ip should be valid IP (see ipaddress)
    try:
        ip_address(row['ip'])
    except ValueError:
        return False

    # time must not be in the future or older than 1 year
    now = pd.Timestamp.now()
    if row['time'] > now:
        return False
    
    # path can't be empty
    if pd.isnull(row['path']) or not row['path'].strip():
        return False

    # status code must be a valid HTTP status code (see http.HTTPStatus)
    if row['status'] not in status_codes:
        return False

    # size can't be negative or empty
    if pd.isnull(row['size']) or row['size'] < 0:
        return False

    return True


def etl(csv_file, db_file):
    df = pd.read_csv(csv_file, parse_dates=['time'])

    bad_rows = df[~df.apply(is_valid_row, axis=1)]
    if len(bad_rows) > 0:
        percent_bad = len(bad_rows)/len(df) * 100
        print(f'{len(bad_rows)} ({percent_bad:.2f}%) bad rows')
        if percent_bad >= max_bad_percent:
            raise ValueError('too many bad rows ({precent_bad:.2f}%)')

    df = df[~df.index.isin(bad_rows.index)]
    with closing(sqlite3.connect(db_file)) as conn:
        conn.execute('BEGIN')
        with conn:
            df.to_sql('traffic', conn, if_exists='append', index=False)

if __name__ == '__main__':
    etl('traffic.csv', 'traffic.db')